In [1]:
import pandas as pd
import collections
import dim_gestor
import etl_gestores
import etl_saresp

from util import gera_id_gestao

In [2]:
#%%time
saresp = etl_saresp.etl()
df_gestao = etl_gestores.etl()
df_gestao = df_gestao.drop(['id_escola','ano_gestao'], axis=1)
mergeidesp = pd.merge(df_gestao, saresp, on='id_gestao', how='left')

In [4]:
mergeidesp = mergeidesp.dropna(subset=['nota_saresp'])
mergeidesp.ano_gestao = mergeidesp.ano_gestao.astype(int)
mergeidesp.id_escola = mergeidesp.id_escola.astype(int)

In [5]:
nota_gestor = {}
nota_gestor['id_gestor'] = []
nota_gestor['nota_acumulada'] = []
nota_gestor['media_nota'] = []
nota_gestor['ano'] = []

for nome, dados in mergeidesp.groupby('id_gestor'):
    lista_anos = dados.ano_gestao.tolist()
    lista_anos = list(set(lista_anos))
    lista_anos.sort()
    nota_acumulada = 0
    accum = 0
    for ano in lista_anos:
        dados_ano = dados[dados.ano_gestao == ano]
        media_ano = 0
        accum_ano = 0
        for escola in set(dados_ano.id_escola):
            dados_ano_escola = dados_ano[dados_ano.id_escola == escola]
            media_ano += dados_ano_escola.nota_saresp.iloc[0]
            accum_ano += 1
        media_ano /= accum_ano

        nota_gestor['id_gestor'].append(nome)
        nota_gestor['ano'].append(ano)
        nota_gestor['nota_acumulada'].append(nota_acumulada / (1 if accum == 0 else accum))
        nota_gestor['media_nota'].append(media_ano)
        nota_acumulada += media_ano
        accum += 1
nota_gestor = pd.DataFrame(nota_gestor)

In [6]:
nota_gestor = nota_gestor.drop(nota_gestor[nota_gestor.nota_acumulada == 0].index)

In [7]:
(nota_gestor.media_nota - nota_gestor.nota_acumulada).describe(percentiles=[0.1,0.9])

count    95333.000000
mean         4.373271
std         15.458966
min       -136.150000
10%        -13.850000
50%          4.600000
90%         22.187500
max        138.950000
dtype: float64

In [8]:
nota_gestor.loc[:,['nota_acumulada','media_nota']].corr()

,nota_acumulada,media_nota
nota_acumulada,1.00000,0.81593
media_nota,0.81593,1.00000


In [9]:
df_gestao = nota_gestor.copy()

# ADICAO DE INFORMACOES SOBRE FALTAS

In [10]:
arquivo = 'D:\\DESAFIO_SEDUC\\dados-VERSAO_1\\ausncias-por-servidor-All-2020-04-13_1011\\BASE_AUSENCIAS_1118.csv'
ausen1118 = pd.read_csv(arquivo, sep=';')
arquivo = 'D:\\DESAFIO_SEDUC\\dados-VERSAO_1\\ausncias-por-servidor-All-2020-04-13_1011\\BASE_AUSENCIAS_0419.csv'
ausen0419 = pd.read_csv(arquivo, sep=';')
arquivo = 'D:\\DESAFIO_SEDUC\\dados-VERSAO_1\\ausncias-por-servidor-All-2020-04-13_1011\\BASE_AUSENCIAS_1119.csv'
ausen1119 = pd.read_csv(arquivo, sep=';')
ausen1118.columns = ausen1119.columns
ausen0419.columns = ausen1119.columns
ausen = ausen1118.append(ausen0419).append(ausen1119)

In [11]:
CATEGORIAS = ['TT_DIAS_FALTA_MEDICA', 'TT_DIAS_FALTA_JUST', 'TT_DIAS_FALTA_INJUST',
              'TT_DIAS_FALTA_ABONADA', 'TT_DIAS_LIC_PREMIO', 'TT_DIAS_LIC_GESTANTE', 
              'TT_DIAS_LIC_ACID_TRAB', 'TT_DIAS_LIC_INTER_PARTIC']

aus = {}
aus['id_gestor'] = []
aus['dias'] = []
for _, row in ausen.iterrows():
    try:
        id_interno = int(row.loc['id_interno'])
    except:
        continue
        
    dias = 0
    for cat in CATEGORIAS:
        try:
            dia_x = int(row.loc[cat])
        except:
            dia_x = 0
        dias += dia_x
    aus['id_gestor'].append(id_interno)
    aus['dias'].append(dias)

aus = pd.DataFrame(aus)

aus2 = {}
aus2['id_gestor'] = []
aus2['teve_falta'] = []
teve_falta = lambda x: 1 if x > 0 else 0
for nome, dados in aus.groupby('id_gestor'):
    aus2['id_gestor'].append(nome)
    aus2['teve_falta'].append(teve_falta(dados.dias.sum()))
    
aus = pd.DataFrame(aus2)    

In [12]:
gest_falta = pd.merge(df_gestao, aus, on='id_gestor',how='left')
gest_falta.teve_falta = gest_falta.teve_falta.fillna(0)

In [13]:
gest_falta.loc[:,['nota_var','teve_falta']].corr()  # correlacao para mostrar que influencia

C:\Users\admprime\Miniconda3\envs\fklearn\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,nota_var,teve_falta
nota_var,NaN,NaN
teve_falta,NaN,1.0


# ADICAO DA FORMACAO

In [14]:
arquivo = 'D:\\DESAFIO_SEDUC\\dados-VERSAO_1\\formao-por-servidor-All-2020-04-13_1013\\BASE_FORMACAO_0419a.csv'
formac0419 = pd.read_csv(arquivo, sep=';', encoding ='latin-1')
arquivo = 'D:\\DESAFIO_SEDUC\\dados-VERSAO_1\\formao-por-servidor-All-2020-04-13_1013\\BASE_FORMACAO_1118.csv'
formac1118 = pd.read_csv(arquivo, sep=';', encoding ='latin-1')
arquivo = 'D:\\DESAFIO_SEDUC\\dados-VERSAO_1\\formao-por-servidor-All-2020-04-13_1013\\BASE_FORMACAO_1119a.csv'
formac1119a = pd.read_csv(arquivo, sep=';', encoding ='latin-1')
formac = formac0419.append(formac1118).append(formac1119a)

C:\Users\admprime\Miniconda3\envs\fklearn\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [15]:
def pontua_formacao(formacao):
    # graduacao/especializacao + 1
    # doutorado +2
    
    if formacao == 'LICENCIATURA':
        return 1
    elif formacao == 'LICENCIATURA + BACHARELADO/TECNÌÒLOGO + ESPECIALIZAÌàÌÄO':
        return 3
    elif formacao == 'LICENCIATURA + ESPECIALIZAÌàÌÄO':
        return 2
    elif formacao == 'ENSINO MÌäDIO':
        return 0
    elif formacao == 'LICENCIATURA + BACHARELADO/TECNÌÒLOGO':
        return 2
    elif formacao == 'BACHARELADO/TECNÌÒLOGO':
        return 1
    elif formacao == 'BACHARELADO/TECNÌÒLOGO':
        return 1
    elif formacao == 'S/INFO':
        return 1
    elif formacao == 'LICENCIATURA + ESPECIALIZAÌàÌÄO + MESTRADO':
        return 3
    elif formacao == 'LICENCIATURA + MESTRADO':
        return 2
    elif formacao == 'LICENCIATURA + BACHARELADO/TECNÌÒLOGO + MESTRADO':
        return 3
    elif formacao == 'ENSINO FUNDAMENTAL':
        return 0
    elif formacao == 'LICENCIATURA + BACHARELADO/TECNÌÒLOGO + ESPECIALIZAÌàÌÄO + MESTRADO':
        return 4
    elif formacao == 'LICENCIATURA + MESTRADO+DOUTORADO':
        return 4
    elif formacao == 'LICENCIATURA + BACHARELADO/TECNÌÒLOGO + MESTRADO + DOUTORADO':
        return 5
    elif formacao == 'APERF/EXTENSÌÄO CULTURAL':
        return 0
    elif formacao == 'LICENCIATURA + ESPECIALIZAÌàÌÄO + MESTRADO + DOUTORADO':
        return 5
    elif formacao == 'BACHARELADO/TECNÌÒLOGO + ESPECIALIZAÌàÌÄO':
        return 2
    elif formacao == 'LICENCIATURA + DOUTORADO':
        return 3
    elif formacao == 'LICENCIATURA + ESPECIALIZAÌàÌÄO+DOUTORADO':
        return 4
    elif formacao == 'BACHARELADO/TECNÌÒLOGO + MESTRADO':
        return 2
    elif formacao == 'LICENCIATURA + BACHARELADO/TECNÌÒLOGO+ESPECIALIZAÌàÌÄO+DOUTORADO':
        return 5
    elif formacao == 'ESPECIALIZAÌàÌÄO':
        return 2
    elif formacao == 'BACHARELADO/TECNÌÒLOGO + MESTRADO + DOUTORADO':
        return 4
    elif formacao == 'LICENCIATURA + BACHARELADO/TECNÌÒLOGO + DOUTORADO':
        return 4
    elif formacao == 'LICENCIATURA + BACHARELADO/TECNÌÒLOGO+ESPECIALIZAÌàÌÄO + MESTRADO + DOUTORADO':
        return 6
    elif formacao == 'ESPECIALIZAÌàÌÄO+MESTRADO':
        return 3
    elif formacao == 'BACHARELADO/TECNÌÒLOGO + ESPECIALIZAÌàÌÄO + MESTRADO':
        return 3
    elif formacao == 'BACHARELADO/TECNÌÒLOGO + DOUTORADO':
        return 3
    elif formacao == 'MESTRADO':
        return 2
    else:
        return 1


In [16]:
formac['formacao_pontos'] = formac.FORMACAO.apply(pontua_formacao)
formac = formac.drop(formac[formac.id_interno.isnull()].index)
formac.id_interno = formac.id_interno.astype(int)
formac = formac.rename({'id_interno': 'id_gestor'}, axis=1)
formac = formac.loc[:,['id_gestor','formacao_pontos']]

In [17]:
formac2 = {}
formac2['id_gestor'] = []
formac2['formacao_pontos'] = []
for nome, dados in formac.groupby('id_gestor'):
    formac2['id_gestor'].append(nome)
    formac2['formacao_pontos'].append(dados.formacao_pontos.max())
    
formac = pd.DataFrame(formac2)

In [18]:
gest = pd.merge(gest_falta, formac, on='id_gestor',how='left')
gest.formacao_pontos = gest.formacao_pontos.fillna(1)

In [19]:
gest.loc[:,['nota_var','formacao_pontos']].corr()

,nota_var,formacao_pontos
nota_var,NaN,NaN
formacao_pontos,NaN,1.0


# ADICIONAR SEXO, IDADE E TEMPO DE TRABALHO

In [20]:
arquivo = 'dados-VERSAO_1\servidores-ativos-por-unidade-All-2020-04-13_1012\BASE_SERVIDORES_ATIVOS_0419.csv'
servidor0419 = pd.read_csv(arquivo, sep=';')
arquivo = 'dados-VERSAO_1\servidores-ativos-por-unidade-All-2020-04-13_1012\BASE_SERVIDORES_ATIVOS_1118.csv'
servidor1118 = pd.read_csv(arquivo, sep=';')
arquivo = 'dados-VERSAO_1\servidores-ativos-por-unidade-All-2020-04-13_1012\BASE_SERVIDORES_ATIVOS_1119.csv'
servidor1119 = pd.read_csv(arquivo, sep=';')
servidor = servidor0419.append(servidor1118).append(servidor1119)
servidor = servidor.drop(servidor[servidor.id_interno.isnull()].index)

C:\Users\admprime\Miniconda3\envs\fklearn\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (11,36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
servidor.id_interno = servidor.id_interno.astype(int)
servidor['SEXO_MASC'] = servidor.SEXO.apply(lambda x: 1 if x == 'M' else 0)

In [22]:
serv = {}
serv['id_gestor'] = []
serv['idade'] = []
serv['anos_trabalho'] = []
serv['sexo'] = []
for nome, dados in servidor.groupby('id_interno'):
    serv['id_gestor'].append(nome)
    serv['idade'].append(dados.IDADE.max())
    serv['anos_trabalho'].append(dados.ANOS_TRAB_CARGO_C.max())
    serv['sexo'].append(dados.SEXO_MASC.max())
serv = pd.DataFrame(serv)

In [23]:
gest_serv = pd.merge(gest, serv, on='id_gestor',how='left')

In [24]:
gest_serv.loc[:,['nota_var','idade','anos_trabalho','sexo']].corr()

,nota_var,idade,anos_trabalho,sexo
nota_var,NaN,NaN,NaN,NaN
idade,NaN,1.000000,0.424805,-0.081001
anos_trabalho,NaN,0.424805,1.000000,-0.089758
sexo,NaN,-0.081001,-0.089758,1.000000


In [25]:
gest_serv = gest_serv.drop(gest_serv[gest_serv.idade.isnull()].index)

In [26]:
gest_serv.idade.describe(percentiles=[0.1,0.9])

count    62589.000000
mean        48.786816
std          7.936413
min         26.000000
10%         38.000000
50%         49.000000
90%         60.000000
max         73.000000
Name: idade, dtype: float64

In [27]:
gest_serv.anos_trabalho.describe(percentiles=[0.25,0.75])

count    62589.000000
mean        13.781303
std          6.326430
min          0.000000
25%         11.000000
50%         13.000000
75%         18.000000
max         46.000000
Name: anos_trabalho, dtype: float64

In [28]:
gest_serv[gest_serv.anos_trabalho <= 8].nota_var.mean()

AttributeError: 'DataFrame' object has no attribute 'nota_var'

In [29]:
gest_serv[(gest_serv.anos_trabalho > 8) &
         (gest_serv.anos_trabalho <= 13)].nota_var.mean()

AttributeError: 'DataFrame' object has no attribute 'nota_var'

In [30]:
gest_serv[(gest_serv.anos_trabalho > 13) &
         (gest_serv.anos_trabalho <= 17)].nota_var.mean()

AttributeError: 'DataFrame' object has no attribute 'nota_var'

In [31]:
gest_serv[gest_serv.anos_trabalho > 17].nota_var.mean()

AttributeError: 'DataFrame' object has no attribute 'nota_var'

In [32]:
gest_serv.sexo = gest_serv.sexo.fillna(1)
gest_serv.idade = gest_serv.idade.fillna(gest_serv.idade.median())
gest_serv.anos_trabalho = gest_serv.anos_trabalho.fillna(gest_serv.anos_trabalho.median())

In [33]:
gest_serv.loc[:,['nota_var','idade','anos_trabalho','sexo']].corr()

,nota_var,idade,anos_trabalho,sexo
nota_var,NaN,NaN,NaN,NaN
idade,NaN,1.000000,0.424805,-0.081001
anos_trabalho,NaN,0.424805,1.000000,-0.089758
sexo,NaN,-0.081001,-0.089758,1.000000


# SALVANDO DIMENSAO DE GESTORES

In [34]:
gest_serv.to_csv('avaliacao_gestores.csv', index=False, sep=';')

In [ ]:
df_gestor = pd.read_csv('avaliacao_gestores.csv',sep=';')
media_nota = df_gestor.media_nota.tolist()
nota_acumulada = df_gestor.nota_acumulada.tolist()
df_gestor = df_gestor.astype(int)
df_gestor['media_nota'] = media_nota
df_gestor['nota_acumulada'] = nota_acumulada

In [ ]:
df_gestor.corr().loc['nota_acumulada', 'media_nota']

In [ ]:
gestor = df_gestor.drop_duplicates(subset='id_gestor').copy()

In [ ]:
gestor.corr()